In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.backend as K
import random
from scipy.optimize import \
    fmin_l_bfgs_b  # https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.fmin_l_bfgs_b.html
from tensorflow.keras.applications import vgg19
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import warnings

from PIL import Image

tf.compat.v1.disable_eager_execution()

random.seed(1618)
np.random.seed(1618)
tf.random.set_seed(1618)

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

CONTENT_IMG_PATH = "/content/drive/MyDrive/Colab Notebooks/content.jpg"
STYLE_IMG_PATH = "/content/drive/MyDrive/Colab Notebooks/the_persistence_of_memory.jpg"

CONTENT_IMG_H = 500
CONTENT_IMG_W = 500

STYLE_IMG_H = 500
STYLE_IMG_W = 500

CONTENT_WEIGHT = 0.1
STYLE_WEIGHT = 1.0
TOTAL_WEIGHT = 1.0

TRANSFER_ROUNDS = 3

numFilters = 3

def deprocessImage(x):
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    # 'BGR'->'RGB'
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')
    return x


def gramMatrix(x):
    features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
    gram = K.dot(features, K.transpose(features))
    return gram


def styleLoss(style, gen):
    return K.sum(K.square(gramMatrix(style) - gramMatrix(gen))) / (
                4. * (numFilters**2) * ((STYLE_IMG_H * STYLE_IMG_W)**2))


def contentLoss(content, gen):
    return K.sum(K.square(gen - content))


def totalLoss(x):
    a = K.square(x[:, : CONTENT_IMG_W - 1, : CONTENT_IMG_H - 1, :] - x[:, 1:, : CONTENT_IMG_W - 1, :])
    b = K.square(x[:, : CONTENT_IMG_W - 1, : CONTENT_IMG_H - 1, :] - x[:, : CONTENT_IMG_W - 1, 1:, :])
    return K.sum(K.pow(a + b, 1.25))


def getRawData():
    print("   Loading images.")
    print("      Content image URL:  \"%s\"." % CONTENT_IMG_PATH)
    print("      Style image URL:    \"%s\"." % STYLE_IMG_PATH)
    cImg = load_img(CONTENT_IMG_PATH)
    tImg = cImg.copy()
    sImg = load_img(STYLE_IMG_PATH)
    print("      Images have been loaded.")
    return (
    (cImg, CONTENT_IMG_H, CONTENT_IMG_W), (sImg, STYLE_IMG_H, STYLE_IMG_W), (tImg, CONTENT_IMG_H, CONTENT_IMG_W))


def preprocessData(raw):
    img, ih, iw = raw
    img = img_to_array(img)
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        img = img.reshape(ih, iw, 3)
    img = img.astype("float64")
    img = np.expand_dims(img, axis=0)
    img = vgg19.preprocess_input(img)
    return img


def styleTransfer(cData, sData, tData):
    print("   Building transfer model.")
    contentTensor = K.variable(cData)
    styleTensor = K.variable(sData)
    genTensor = K.placeholder((1, CONTENT_IMG_H, CONTENT_IMG_W, 3))

    inputTensor = K.concatenate([contentTensor, styleTensor, genTensor], axis=0)
    model = vgg19.VGG19(include_top=False, weights="imagenet", input_tensor=inputTensor)

    outputDict = dict([(layer.name, layer.output) for layer in model.layers])
    print("   VGG19 model loaded.")

    loss = 0.0
    styleLayerNames = ["block1_conv1", "block2_conv1", "block3_conv1", "block4_conv1", "block5_conv1"]
    contentLayerName = "block5_conv2"

    print("   Calculating content loss.")
    contentLayer = outputDict[contentLayerName]
    contentOutput = contentLayer[0, :, :, :]
    genOutput = contentLayer[2, :, :, :]
    loss += CONTENT_WEIGHT * contentLoss(contentOutput, genOutput)

    print("   Calculating style loss.")
    for layerName in styleLayerNames:
        styleLayer = outputDict[layerName]
        styleOutput = styleLayer[1, :, :, :]
        styleGen = styleLayer[2, :, :, :]
        sLoss = styleLoss(styleOutput, styleGen)
        loss += (STYLE_WEIGHT / len(styleLayerNames)) * sLoss
    loss += TOTAL_WEIGHT * totalLoss(genTensor)
    grads = K.gradients(loss, genTensor)[0]

    x = tData
    x = x.flatten()
    kFunction = K.function([genTensor], [loss, grads])

    class Test:
        def __init__(self):
            self.loss = None
            self.grad = None

        def transferLoss(self, x):
            x = x.reshape(genTensor.shape)
            outs = kFunction([x])
            transLoss = outs[0]
            grad = outs[1].flatten().astype('float64')
            self.loss = transLoss
            self.grad = grad
            return self.loss

        def gradFunction(self, x):
            grad = np.copy(self.grad)
            self.loss = None
            grad = None
            return self.grad

    test = Test()

    print("   Beginning transfer.")
    for i in range(TRANSFER_ROUNDS):
        print("   Step %d." % i)
        x, tLoss, d = fmin_l_bfgs_b(test.transferLoss, x, fprime=test.gradFunction, maxiter=20)
        print("      Loss: %f." % tLoss)
        img = x.copy().reshape((CONTENT_IMG_H, CONTENT_IMG_W, 3))
        img = deprocessImage(img)
        saveFile = "/content/drive/MyDrive/Colab Notebooks/output" + str(i) + ".jpg"
        Image.fromarray(img).save(saveFile)
        print("      Image saved to \"%s\"." % saveFile)
    print("   Transfer complete.")


def main():
    print("Starting style transfer program.")
    raw = getRawData()
    cData = preprocessData(raw[0])
    sData = preprocessData(raw[1])
    tData = preprocessData(raw[2])
    styleTransfer(cData, sData, tData)
    print("Done. Goodbye.")


if __name__ == "__main__":
    main()


Starting style transfer program.
   Loading images.
      Content image URL:  "/content/drive/MyDrive/Colab Notebooks/content.jpg".
      Style image URL:    "/content/drive/MyDrive/Colab Notebooks/the_persistence_of_memory.jpg".
      Images have been loaded.
   Building transfer model.
   VGG19 model loaded.
   Calculating content loss.
   Calculating style loss.
   Beginning transfer.
   Step 0.
      Loss: 1286244608.000000.
      Image saved to "/content/drive/MyDrive/Colab Notebooks/output0.jpg".
   Step 1.
      Loss: 962892416.000000.
      Image saved to "/content/drive/MyDrive/Colab Notebooks/output1.jpg".
   Step 2.
      Loss: 850125696.000000.
      Image saved to "/content/drive/MyDrive/Colab Notebooks/output2.jpg".
   Transfer complete.
Done. Goodbye.
